In [1]:
import pickle
import gc
from datetime import datetime
from os import mkdir
from os.path import join, exists

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

from sklearn import metrics

In [2]:
train_f = pd.read_pickle('../dataset/train/train_all_feature.pkl')
train_f.shape

(879539, 438)

In [3]:
test_f = pd.read_pickle('../dataset/train/test_all_feature.pkl')
test_f.shape

(215287, 438)

In [4]:
model_path = '../dataset/lightgbmmodel.pkl'

In [5]:
luck = 123456

In [6]:
model = LGBMClassifier(nthread=4, silent=False, objective= 'binary', seed=luck)

In [7]:
parameters = {
#     'gamma': [0.05],
    #     'n_estimators': [800],
    'max_depth': [2, 6, 12],
#     'max_depth': [12],
#     'num_leaves': range(30, 50, 4), 

#     'learning_rate': [0.15, 0.3],
#         'subsample': [0.9],
#         'colsample_bytree': [0.9],
    #     'reg_alpha': [0, 1, 5],
    #     'reg_lambda': [0, 1, 4],
}

In [8]:
use_columns = [
    x for x in train_f.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid',
        'orderlabel', 'index'
    ]
]

In [9]:
cv = model_selection.ShuffleSplit(n_splits=1, test_size=0.2, random_state=luck)
clf = model_selection.GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=True)

In [10]:
clf_feture = train_f.sample(1000, random_state=luck)

In [11]:
%time clf.fit(clf_feture[use_columns], clf_feture['orderlabel'])

Fitting 1 folds for each of 3 candidates, totalling 3 fits
CPU times: user 262 ms, sys: 43.8 ms, total: 306 ms
Wall time: 757 ms


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.5s finished


GridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=123456, test_size=0.2, train_size=None),
       error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=4,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=123456, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [2, 6, 12]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=True)

In [12]:
clf.best_score_

0.96499999999999997

In [13]:
clf.best_params_

{'max_depth': 2}

In [14]:
# # clf.best_params_['n_estimators'] = 
clf.best_params_['max_depth'] = 6
# clf.best_params_['reg_lambda'] = 0.5
clf.best_params_['subsample'] = 0.8
clf.best_params_['colsample_bytree'] = 0.8

clf.best_params_['colsample_bylevel'] = 0.7

# clf.best_params_['n_estimators'] = 1000
clf.best_params_['learning_rate'] = 0.1
clf.best_params_['num_leaves'] = 56


# clf.best_params_['reg_alpha'] = 1
# clf.best_params_['min_data_in_leaf'] = 3

# clf.best_params_['min_child_weight'] = 1
clf.best_params_['lambda_l2'] = 10
clf.best_params_['tree_method'] = 'exact'
clf.best_params_['metric'] = 'binary_logloss'



In [15]:
2**6

64

In [18]:
params = {
    'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'binary_logloss',
        'metric': 'auc',
    'min_child_weight': 1.5,
#     'num_leaves': 2 ** 7,
    'max_depth':6,
      'num_leaves':32,
    'lambda_l2': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
#     'colsample_bylevel': 0.7,
    'learning_rate': 0.05,
#     'tree_method': 'exact',
}

In [19]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=1300,
    **params)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, lambda_l2=10,
        learning_rate=0.05, max_bin=255, max_depth=6, metric='auc',
        min_child_samples=10, min_child_weight=1.5, min_split_gain=0,
        n_estimators=1300, nthread=4, num_leaves=32, objective='binary',
        reg_alpha=0, reg_lambda=0, seed=123456, silent=False,
        subsample=0.7, subsample_for_bin=50000, subsample_freq=1)

In [20]:
# categorical_feature=['order_' + x for x in  ['roomservice_8','roomservice_4','roomservice_3','roomtag_1']]

In [21]:
model.fit(
    train_f[use_columns],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns],
    train_f['orderlabel']),
             (test_f[use_columns],
    test_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
    early_stopping_rounds=50,
)

[1]	valid_0's auc: 0.856782	valid_1's auc: 0.855211
Train until valid scores didn't improve in 50 rounds.
[2]	valid_0's auc: 0.870396	valid_1's auc: 0.870085
[3]	valid_0's auc: 0.890656	valid_1's auc: 0.890188
[4]	valid_0's auc: 0.892694	valid_1's auc: 0.891979
[5]	valid_0's auc: 0.893032	valid_1's auc: 0.892313
[6]	valid_0's auc: 0.894197	valid_1's auc: 0.893787
[7]	valid_0's auc: 0.897056	valid_1's auc: 0.896576
[8]	valid_0's auc: 0.896783	valid_1's auc: 0.896372
[9]	valid_0's auc: 0.896953	valid_1's auc: 0.89645
[10]	valid_0's auc: 0.897521	valid_1's auc: 0.897246
[11]	valid_0's auc: 0.897721	valid_1's auc: 0.89762
[12]	valid_0's auc: 0.902054	valid_1's auc: 0.901459
[13]	valid_0's auc: 0.902372	valid_1's auc: 0.901671
[14]	valid_0's auc: 0.902745	valid_1's auc: 0.901915
[15]	valid_0's auc: 0.904261	valid_1's auc: 0.903536
[16]	valid_0's auc: 0.904343	valid_1's auc: 0.903707
[17]	valid_0's auc: 0.904988	valid_1's auc: 0.904138
[18]	valid_0's auc: 0.904892	valid_1's auc: 0.903989
[19

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, lambda_l2=10,
        learning_rate=0.05, max_bin=255, max_depth=6, metric='auc',
        min_child_samples=10, min_child_weight=1.5, min_split_gain=0,
        n_estimators=1300, nthread=4, num_leaves=32, objective='binary',
        reg_alpha=0, reg_lambda=0, seed=123456, silent=False,
        subsample=0.7, subsample_for_bin=50000, subsample_freq=1)

In [22]:
model.best_iteration

872

In [23]:
def get_score(model, train_f):
    result = model.predict_proba(train_f[use_columns])[:, 1]
    train_f['prob'] = result
    train_x = train_f[['orderid', 'prob', 'orderlabel']]
    train_x.sort_values('prob', ascending=False, inplace=True)
    finall_result_x = train_x.drop_duplicates(['orderid'])
    train_score = finall_result_x.orderlabel.mean()
    print(train_score)
    return train_score

In [24]:
train_score = get_score(model, train_f)

0.48724562912


/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
test_score = get_score(model, test_f)

0.440255695788


/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
acu = model.score(test_f[use_columns], test_f['orderlabel'])
acu

0.97444341739166784

In [44]:
# y_predict = model.predict_proba(train_features[use_columns])[:, 1]
# fpr, tpr, thresholds  = metrics.roc_curve(train_features['orderlabel'], y_predict, pos_label=1)
# auc_score = metrics.auc(fpr, tpr)
# auc_score

In [27]:
if not exists('models'):
    mkdir('models')
model_importance_path = join('models', datetime.now().strftime('%d-%H%M-importance.txt'))
model_importance_path

'models/06-2150-importance.txt'

In [28]:
importance_df = pd.DataFrame(model.feature_importances_, index=use_columns)

importance_df.sort_values(0, ascending=False, inplace=True)
importance_df.to_csv(model_importance_path)

In [29]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in clf.best_params_.items()])
parms

'learning_rate: 0.1,colsample_bylevel: 0.7,subsample: 0.8,lambda_l2: 10,num_leaves: 56,metric: binary_logloss,colsample_bytree: 0.8,tree_method: exact,max_depth: 6'

In [30]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in params.items()])
parms

'min_child_weight: 1.5,learning_rate: 0.05,subsample: 0.7,num_leaves: 32,boosting_type: gbdt,lambda_l2: 10,metric: auc,colsample_bytree: 0.7,max_depth: 6'

In [31]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.best_score['valid_0']['auc'],
    'model_test',
    model.best_score['valid_1']['auc'],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    'shape', train_f.shape,
    end='\n',
    sep='  ')

06-21-50:  online  ??  test_score  0.440255695788  train_score  0.48724562912  model_train  0.956333556711  model_test  0.935368984433  accu  0.974443417392  n  872  params  min_child_weight: 1.5,learning_rate: 0.05,subsample: 0.7,num_leaves: 32,boosting_type: gbdt,lambda_l2: 10,metric: auc,colsample_bytree: 0.7,max_depth: 6  shape  (879539, 439)


In [32]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.best_score['valid_0']['auc'],
    'model_test',
    model.best_score['valid_1']['auc'],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    'shape', train_f.shape,
    end='\n',
    sep='  ', file=open('lgb_result.txt', 'a+'))

In [33]:
del train_f, test_f

In [34]:
gc.collect()

88

In [35]:
train_f = pd.read_pickle('../dataset/train/all_feature.pkl')
train_f.shape

(1094826, 438)

In [36]:
n_est = model.best_iteration + 200 if model.best_iteration > 0 else 1500

In [37]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=n_est,
    **params)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, lambda_l2=10,
        learning_rate=0.05, max_bin=255, max_depth=6, metric='auc',
        min_child_samples=10, min_child_weight=1.5, min_split_gain=0,
        n_estimators=1072, nthread=4, num_leaves=32, objective='binary',
        reg_alpha=0, reg_lambda=0, seed=123456, silent=False,
        subsample=0.7, subsample_for_bin=50000, subsample_freq=1)

In [38]:
# categorical_feature=['order_' + x for x in  ['roomservice_8','roomservice_4','roomservice_3','roomtag_1']]

In [39]:
model.fit(
    train_f[use_columns],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns],
    train_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
)

[1]	valid_0's auc: 0.879326
[2]	valid_0's auc: 0.888655
[3]	valid_0's auc: 0.892491
[4]	valid_0's auc: 0.896299
[5]	valid_0's auc: 0.898134
[6]	valid_0's auc: 0.898313
[7]	valid_0's auc: 0.898305
[8]	valid_0's auc: 0.898866
[9]	valid_0's auc: 0.899023
[10]	valid_0's auc: 0.899232
[11]	valid_0's auc: 0.902061
[12]	valid_0's auc: 0.905073
[13]	valid_0's auc: 0.905231
[14]	valid_0's auc: 0.905083
[15]	valid_0's auc: 0.905721
[16]	valid_0's auc: 0.905821
[17]	valid_0's auc: 0.906333
[18]	valid_0's auc: 0.906374
[19]	valid_0's auc: 0.906646
[20]	valid_0's auc: 0.907777
[21]	valid_0's auc: 0.90839
[22]	valid_0's auc: 0.908992
[23]	valid_0's auc: 0.909131
[24]	valid_0's auc: 0.909133
[25]	valid_0's auc: 0.909181
[26]	valid_0's auc: 0.909229
[27]	valid_0's auc: 0.909256
[28]	valid_0's auc: 0.909373
[29]	valid_0's auc: 0.909423
[30]	valid_0's auc: 0.909739
[31]	valid_0's auc: 0.91106
[32]	valid_0's auc: 0.911115
[33]	valid_0's auc: 0.911145
[34]	valid_0's auc: 0.91205
[35]	valid_0's auc: 0.9120

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, lambda_l2=10,
        learning_rate=0.05, max_bin=255, max_depth=6, metric='auc',
        min_child_samples=10, min_child_weight=1.5, min_split_gain=0,
        n_estimators=1072, nthread=4, num_leaves=32, objective='binary',
        reg_alpha=0, reg_lambda=0, seed=123456, silent=False,
        subsample=0.7, subsample_for_bin=50000, subsample_freq=1)

In [42]:
pickle.dump(model, open(model_path, 'wb'))

In [41]:
# clf1 =  LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=42, max_depth=6)
# clf2 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=32, max_depth=5)
# clf3 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=27, max_depth=3)
# lr = LogisticRegression()
# sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3], 
#                           meta_classifier=lr)

# model = sclf